In [ ]:
# BdG solver (1D, s-wave, mean-field self-consistent)
using LinearAlgebra, Plots, LaTeXStrings, Statistics, Random # Necessary packages
# Run in Google Colab if Julia is not locally installed

In [3]:
# 1. Assemble the BdG Hamiltonian from its components

# Build kinetic (tight-binding) Hamiltonian with open boundary conditions
function build_kinetic(N_sites::Int, t::Float64)
    # Compact form for a tridiagonal matrix (OBC)
    return -t * diagm(1 => ones(N_sites-1), -1 => ones(N_sites-1))
end

function build_BdG_hamiltonian(h_kinetic, μ, Δ)
    N_sites = size(h_kinetic, 1)
    Id = I(N_sites)
    H11 = h_kinetic .- μ .* Id
    
    # Assemble using Julia's block matrix syntax for clarity
    H = [ Matrix(H11)      Diagonal(Δ);
          Diagonal(conj.(Δ))  -Matrix(H11) ]
    return H
end

build_BdG_hamiltonian (generic function with 1 method)

In [ ]:
# 2. Compute the new order parameter from the eigenstates (corrected BdG formulation)
function update_delta(vals, vecs, N_sites, g, T)
    Δ_new = zeros(ComplexF64, N_sites)
    
    # At T=0: sum only over negative energy states (occupied)
    # The key is: Δ_i = -g * Σ_{E_n<0} u_n(i) * v_n*(i)
    @inbounds for n in 1:length(vals)
        E_n = vals[n]
        
        # At T=0, only negative energy states are occupied
        if T == 0.0
            if E_n >= 0.0
                continue  # Skip positive energy states
            end
            weight = 1.0
        else
            # At finite T, use Fermi-Dirac distribution
            f_n = 1.0 / (exp(E_n/T) + 1.0)
            if f_n < 1e-12
                continue
            end
            weight = f_n
        end
        
        # Extract u and v components for site i
        # Accumulate contribution: 
        
        psi = @view vecs[:, n]               
        @inbounds for i in 1:N_sites
            Δ_new[i] += weight * psi[i] * conj(psi[N_sites + i])
        end

    end
    
    # Apply coupling constant: Δ_i = -g * Σ u*v
    return -g .* Δ_new
end

update_delta (generic function with 1 method)

In [ ]:
# --- Main self-consistent solver ---
function solve_BdG(N_sites::Int;
                            t::Float64=1.0, # hopping
                            Δ_init::Vector{ComplexF64}= ones(ComplexF64, N_sites), # initial guess (small!)
                            μ::Float64=0.0, # chemical potential
                            g::Float64=1.0, # interaction strength
                            T::Float64=0.0, # temperature
                            tol::Float64=1e-8,
                            iterations::Int=200,
                            mix::Float64=0.2)

    h_kinetic = build_kinetic(N_sites, t) # Kinetic Hamiltonian
    Δ = copy(Δ_init) # Current order parameter
    Δ_new = similar(Δ) # New order parameter
    
    # Initialize eigenvalues and eigenvectors
    vals = zeros(Float64, 2*N_sites)
    vecs = zeros(ComplexF64, 2*N_sites, 2*N_sites)
    err = 0.0

    @inbounds for it in 1:iterations # Self-consistency loop
        # Build and diagonalize BdG Hamiltonian
        H = build_BdG_hamiltonian(h_kinetic, μ, Δ)
        vals, vecs = eigen(Hermitian(H)).values, eigen(Hermitian(H)).vectors  # preferible separar para no llamar dos veces

        # Compute new order parameter
        Δ_new = update_delta(vals, vecs, N_sites, g, T)

        # Check convergence
        err = maximum(abs.(Δ_new .- Δ))

        # Mixing for stability
        Δ = mix .* Δ_new .+ (1.0 - mix) .* Δ
        
        if err < tol
            println("Converged in $it iterations (err = $err)")
            return Δ, vals, vecs
        end
    end

    @warn "No convergence after $iterations iterations; last err = $err"
    return Δ, vals, vecs
end

solve_BdG (generic function with 1 method)

In [ ]:
function plot_delta_profile(Δ::Vector{ComplexF64}, N_sites::Int64, g::Float64, T::Float64) # Plot and save the gap profile
    plt1 = plot(abs.(Δ), xlabel=L"$i$ site ", ylabel=L"\left| \Delta_i \right|", title = L"BdG Gap profile ($N$ =%$(N_sites), $g$=%$(g), $T$=%$(T))" ,        legend=false, lw=2, color=:blue, grid=false)
    savefig(plt1, "BdG_gap_profile.png")
    println("Saved gap profile: BdG_gap_profile.png")
end

# Compute average gap excluding boundary effects
function compute_average_gap(Δ::Vector{ComplexF64}; boundary_exclude::Int=10)
    N = length(Δ)
    if N <= 2*boundary_exclude
        return mean(abs.(Δ))
    else
        # Exclude boundary sites
        return mean(abs.(Δ[boundary_exclude+1:N-boundary_exclude]))
    end
end

compute_average_gap (generic function with 1 method)

In [ ]:
# Small demo: run solver, plot Δ profile and spectrum
function BdG_demo()
    N_sites = 180
    t = 1.0
    μ = -1.0    # CRITICAL: at half-filling for maximum DOS (Density of states) at Fermi level
    g = 2.5    # Moderate coupling (must be > g_critical ~ 1.0)
    T = 0.0    # T=0 demo

    println("Running BdG self-consistent solver (N_sites=$N_sites, g=$g, T=$T, μ=$μ)")    
    # Initial guess: small uniform value with noise to break symmetry
    Random.seed!(42)
    Δ_init = 0.1* ones(ComplexF64, N_sites) .+ 0.01*randn(ComplexF64, N_sites)

    Δ, vals, vecs = solve_BdG(N_sites; t=t, μ=μ, g=g, T=T, Δ_init=Δ_init, # Function to solve BdG
                              tol=1e-6, mix=0.2, iterations=100)
    
    plot_delta_profile(Δ, N_sites, g, T) # Plot and save gap profile
    
    return Δ, vals, vecs
end

BdG_demo (generic function with 1 method)

In [8]:
BdG_demo()

Running BdG self-consistent solver (N_sites=180, g=2.5, T=0.0, μ=-1.0)


┌ Warning: No convergence after 100 iterations; last err = 0.00021294466793682855
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41


Saved gap profile: BdG_gap_profile.png


(ComplexF64[0.7686234888865895 - 0.014223383623696021im, 0.6936116184840418 - 0.012595369133093552im, 0.6251949648939916 - 0.010984126490652583im, 0.6539007198505697 - 0.010856952872834691im, 0.6464573577007179 - 0.010091116448370808im, 0.643650145029441 - 0.009363651427238124im, 0.6455116934805788 - 0.008704829767015546im, 0.6447614429403976 - 0.008061499581738225im, 0.644656481861924 - 0.007502604481609613im, 0.6447891637195691 - 0.007041433739996355im  …  0.6447817001117722 + 0.007665645725186751im, 0.6446546100971543 + 0.00763350338863644im, 0.6447652684944171 + 0.007724468233570217im, 0.6455213582482819 + 0.007936726734227196im, 0.6436656530280661 + 0.008217324564339629im, 0.6464784797795207 + 0.008627477145729777im, 0.6539270402813938 + 0.009134278427993098im, 0.6252245167009058 + 0.009147866249263456im, 0.693647338002078 + 0.010444361050628338im, 0.7686649586160484 + 0.011768901695492731im], [-3.0682069046518587, -3.067325131079838, -3.0658556793142906, -3.0637991909099322, -3.0

## Transición de fase superconductora

A continuación calculamos el parámetro de orden promedio $\langle |\Delta| \rangle$ como función de la temperatura para observar la **transición de fase superconductora**.

### Física esperada:
- A **T = 0**: $\Delta \neq 0$ (fase superconductora)
- A **T > Tc**: $\Delta \to 0$ (fase normal)
- La transición es de **segundo orden** (continua)

### Implementación:
- **Warm start**: Usamos la solución a temperatura $T_i$ como condición inicial para $T_{i+1}$
- Esto acelera la convergencia y mejora la estabilidad
- También hay versión **paralela** disponible (más rápida, sin warm start)
- El código usa correctamente la distribución de **Fermi-Dirac**:
 $$f(E) = \frac{1}{e^{E/T} + 1}$$

A T=0 solo suma estados ocupados (E<0), a T>0 suma todos con peso $f(E)$.

In [9]:
# Compute phase transition: scan temperature and measure average gap
function compute_multiple_gaps(temperatures; N_sites::Int=180, t::Float64=1.0, 
                               μ::Float64=0.0, g::Float64=1.0, tol::Float64=1e-6, 
                               iterations::Int=100, mix::Float64=0.2,
                               threshold::Float64=1e-3)
                                   
    average_gaps = zeros(Float64, length(temperatures))
    
    Random.seed!(42)
    Δ_seed = 0.1 .* ones(ComplexF64, N_sites) .+ 0.01*randn(ComplexF64, N_sites) 

    for (i, T) in enumerate(temperatures) 
        # Solve BdG at this temperature
        Δ, vals, vecs = solve_BdG(N_sites; t=t, μ=μ, g=g, T=T, Δ_init=Δ_seed,
                                   tol=tol, iterations=iterations, mix=mix)
        
        # Compute average gap (excluding boundaries)
        average_gaps[i] = compute_average_gap(Δ)

        # Warm start: use converged solution as seed for next temperature
        Δ_seed = copy(Δ)
    end

    # Determine Tc: first temperature where <Δ> drops below threshold
    Tc_index = findfirst(x -> x <= threshold, average_gaps)
    Tc = Tc_index === nothing ? temperatures[end] : temperatures[Tc_index]

    return average_gaps, Tc
end

compute_multiple_gaps (generic function with 1 method)

In [ ]:
# Plot phase transition results
function plot_phase_transitions(temperatures, average_gaps, Tc, g, N_sites)
    plt = plot(temperatures, average_gaps, 
               ylabel=L"\langle |\Delta| \rangle", 
               xlabel=L"T / t", 
               title="Superconducting Phase Transition (g=$g, N=$N_sites)",
               label="BdG mean-field",
               lw=2, color=:blue, grid=true,
               marker=:circle, markersize=3)

    # Marcar Tc
    vline!([Tc], label="Tc ≈ $(round(Tc, digits=3))", linestyle=:dash, color=:red, lw=2)

    savefig(plt, "phase_transition.png")
    println("Saved: phase_transition.png")
    return plt
end

In [ ]:
# Function wrapper that computes and plots the phase transition
function compute_phase_transition(temperatures; N_sites::Int=180, t::Float64=1.0, 
                                  μ::Float64=-1.0, g::Float64=1.0, tol::Float64=1e-5, 
                                  iterations::Int=100, mix::Float64=0.2,
                                  threshold::Float64=1e-3)
                                   
    average_gaps, Tc = compute_multiple_gaps(temperatures; N_sites=N_sites, t=t, μ=μ, 
                                             g=g, tol=tol, iterations=iterations, 
                                             mix=mix, threshold=threshold)
    
    plt = plot_phase_transitions(temperatures, average_gaps, Tc, g, N_sites)
    
    return average_gaps, Tc, plt
end

compute_phase_transition (generic function with 1 method)

In [ ]:
# Example: compute phase transition for g=2.5
temperatures = collect(0.0:0.025:1.0)
gaps, Tc, plt = compute_phase_transition(temperatures; N_sites=180, g=2.5, mix=0.3, iterations=100)
#Note: Convergence can be slow near Tc due to criteria of threshold; adjust mix and iterations as needed.

┌ Warning: No convergence after 500 iterations; last err = 2.0893420941955366e-5
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41
┌ Warning: No convergence after 500 iterations; last err = 9.979477386802922e-6
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41
┌ Warning: No convergence after 500 iterations; last err = 6.823489563587593e-6
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41
┌ Warning: No convergence after 500 iterations; last err = 4.827907104625868e-6
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a7

Converged in 326 iterations (err = 9.833665643023607e-7)


┌ Warning: No convergence after 500 iterations; last err = 4.9917390258057054e-5
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41
┌ Warning: No convergence after 500 iterations; last err = 0.000357260757914571
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41
┌ Warning: No convergence after 500 iterations; last err = 4.320595416430902e-5
└ @ Main /home/joseangel/Documents/Areas/Cursos_Maestria/2026-1/QFT_MatCon/Tareas_Extra/Bogoliubov_de_Gennes/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:41


Converged in 238 iterations (err = 9.995089157840893e-7)
Converged in 14 iterations (err = 9.999770546624475e-7)
Converged in 8 iterations (err = 9.807238087063293e-7)
Converged in 5 iterations (err = 9.651147736471476e-7)
Converged in 3 iterations (err = 9.729236154609378e-7)
Converged in 2 iterations (err = 9.899929586300935e-7)
Converged in 2 iterations (err = 9.746101305596578e-7)
Converged in 2 iterations (err = 9.328893493381545e-7)
Converged in 1 iterations (err = 9.412987557772266e-7)
Converged in 1 iterations (err = 9.352612325007616e-7)
Converged in 1 iterations (err = 9.167374656564276e-7)
Converged in 1 iterations (err = 8.87758688073195e-7)
Converged in 1 iterations (err = 8.503417205715432e-7)
Converged in 1 iterations (err = 8.0642113709625e-7)
Converged in 1 iterations (err = 7.577980482913701e-7)
Converged in 1 iterations (err = 7.061042734152078e-7)
Converged in 1 iterations (err = 6.527802103345933e-7)
Converged in 1 iterations (err = 5.990644093277593e-7)
Converged 

([0.6448112053410937, 0.6448112011425667, 0.644810109588528, 0.6447124324129263, 0.6438204888625226, 0.6406859835763367, 0.6337063593198285, 0.6214532885546694, 0.6026193543618233, 0.5758161041640528  …  6.43945056032946e-9, 5.711086570015802e-9, 5.0419378227844035e-9, 4.431585677205523e-9, 3.878586595095885e-9, 3.3806972523314918e-9, 2.9350778917708767e-9, 2.538471585927775e-9, 2.187358503602496e-9, 1.8780857768873184e-9], 0.4, Plot{Plots.GRBackend() n=2})